# 1 Colors_1 classification model

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

### Define image and batch size

In [78]:
IMAGE_SIZE = 90
BATCH_SIZE = 32

### Import train, test, and val

In [79]:
colors_1_train = utils.image_dataset_from_directory("datasets/data/split/colors_1/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_1_val = utils.image_dataset_from_directory("datasets/data/split/colors_1/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_1_test = utils.image_dataset_from_directory("datasets/data/split/colors_1/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 7267 files belonging to 15 classes.
Found 1550 files belonging to 15 classes.
Found 1556 files belonging to 15 classes.


### Augmentation

In [80]:
colors_1_data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
] 
)

### Define Model

In [81]:
colors_1_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

x = colors_1_data_augmentation(colors_1_inputs)

x = layers.Rescaling(1./255)(colors_1_inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) 
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.GlobalAveragePooling2D()(x)

x = layers.Flatten()(x)

colors_1_outputs = layers.Dense(15, activation="softmax")(x)

colors_1_model = keras.Model(inputs=colors_1_inputs, outputs=colors_1_outputs)

colors_1_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

colors_1_model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 90, 90, 3)]       0         
                                                                 
 rescaling_11 (Rescaling)    (None, 90, 90, 3)         0         
                                                                 
 conv2d_52 (Conv2D)          (None, 88, 88, 32)        896       
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 44, 44, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_53 (Conv2D)          (None, 42, 42, 64)        18496     
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 21, 21, 64)       0         
 g2D)                                                     

In [82]:
colors_1_callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="2_firstAugmentation_colors_1_checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

### Run Model

In [83]:
history = colors_1_model.fit(
  colors_1_train.cache(),
  epochs=30,
  callbacks=colors_1_callbacks_list,
  validation_data=colors_1_val.cache()
)

Epoch 1/30


2022-07-30 06:38:29.839116: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


228/228 [==============================] - ETA: 0s - loss: 1.5784 - accuracy: 0.4770

2022-07-30 06:38:38.899058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


228/228 [==============================] - 10s 43ms/step - loss: 1.5784 - accuracy: 0.4770 - val_loss: 1.3032 - val_accuracy: 0.5594
Epoch 2/30
228/228 [==============================] - 9s 39ms/step - loss: 0.9624 - accuracy: 0.6640 - val_loss: 0.9135 - val_accuracy: 0.6606
Epoch 3/30
228/228 [==============================] - 9s 38ms/step - loss: 0.7781 - accuracy: 0.7275 - val_loss: 0.9508 - val_accuracy: 0.6703
Epoch 4/30
228/228 [==============================] - 9s 38ms/step - loss: 0.6702 - accuracy: 0.7680 - val_loss: 0.8391 - val_accuracy: 0.7097
Epoch 5/30
228/228 [==============================] - 9s 39ms/step - loss: 0.5959 - accuracy: 0.7980 - val_loss: 0.7753 - val_accuracy: 0.7471
Epoch 6/30
228/228 [==============================] - 9s 40ms/step - loss: 0.5212 - accuracy: 0.8237 - val_loss: 0.6954 - val_accuracy: 0.7613
Epoch 7/30
228/228 [==============================] - 9s 40ms/step - loss: 0.4605 - accuracy: 0.8462 - val_loss: 0.6076 - val_accuracy: 0.7806
Epoch 8/3

In [84]:
# colors_1_model.evaluate(colors_1_test)

colors_1_model_test = keras.models.load_model("1_base_colors_1_checkpoint_path.keras")
colors_1_model_test.evaluate(colors_1_test)

2022-07-30 06:40:57.645939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


49/49 [==============================] - 1s 19ms/step - loss: 0.6778 - accuracy: 0.7783


[0.6777648329734802, 0.7782776355743408]

# 2 Vehicle Type classification

### Import train, test, and val datasets

In [88]:
vehicle_type_train = utils.image_dataset_from_directory("datasets/data/split/vehicle_type/train/", image_size=(224, 224), batch_size=16)
vehicle_type_val = utils.image_dataset_from_directory("datasets/data/split/vehicle_type/val", image_size=(224, 224), batch_size=16)
vehicle_type_test = utils.image_dataset_from_directory("datasets/data/split/vehicle_type/test", image_size=(224, 224), batch_size=16)

Found 840 files belonging to 3 classes.
Found 180 files belonging to 3 classes.
Found 180 files belonging to 3 classes.


### Augmentation

In [89]:
vehicle_type_data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
] 
)

### Define Model

In [90]:
vehicle_type_inputs = keras.Input(shape=(224, 224, 3))

x = vehicle_type_data_augmentation(vehicle_type_inputs)

x = layers.Rescaling(1./255)(vehicle_type_inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=3, activation="relu")(x)

# x = layers.Flatten()(x)
x = layers.GlobalAveragePooling2D()(x)

vehicle_type_outputs = layers.Dense(3, activation="softmax")(x)

vehicle_type_model = keras.Model(inputs=vehicle_type_inputs, outputs=vehicle_type_outputs)

vehicle_type_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

vehicle_type_model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_13 (Rescaling)    (None, 224, 224, 3)       0         
                                                                 
 conv2d_61 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_48 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_62 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                     

### Callbacks

In [91]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="3_secondAugmentation_vehicle_type_checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

### Run the Model

In [92]:
history = vehicle_type_model.fit(
  vehicle_type_train.cache(),
  epochs=30,
  callbacks=callbacks_list,
  validation_data=vehicle_type_val.cache()
)

Epoch 1/30


2022-07-30 06:42:14.729922: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - ETA: 0s - loss: 1.1868 - accuracy: 0.3607

2022-07-30 06:42:21.876071: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - 8s 139ms/step - loss: 1.1868 - accuracy: 0.3607 - val_loss: 1.1102 - val_accuracy: 0.3833
Epoch 2/30
53/53 [==============================] - 7s 128ms/step - loss: 1.1007 - accuracy: 0.4214 - val_loss: 1.0603 - val_accuracy: 0.4278
Epoch 3/30
53/53 [==============================] - 7s 126ms/step - loss: 1.0410 - accuracy: 0.5512 - val_loss: 0.9326 - val_accuracy: 0.5333
Epoch 4/30
53/53 [==============================] - 7s 126ms/step - loss: 0.9341 - accuracy: 0.6167 - val_loss: 0.9115 - val_accuracy: 0.5056
Epoch 5/30
53/53 [==============================] - 7s 126ms/step - loss: 0.8524 - accuracy: 0.6476 - val_loss: 0.8673 - val_accuracy: 0.5667
Epoch 6/30
53/53 [==============================] - 7s 126ms/step - loss: 0.7628 - accuracy: 0.6952 - val_loss: 0.7332 - val_accuracy: 0.6611
Epoch 7/30
53/53 [==============================] - 7s 131ms/step - loss: 0.6713 - accuracy: 0.7488 - val_loss: 0.5906 - val_accuracy: 0.7500
Epoch 8/30
53/53 

In [94]:
vehicle_types_test = keras.models.load_model("2_firstAugmentation_vehicle_type_checkpoint_path.keras")
vehicle_types_test.evaluate(vehicle_type_test)
# vehicle_type_model.evaluate(vehicle_type_test)

2022-07-30 06:45:09.027585: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 1s 46ms/step - loss: 0.3933 - accuracy: 0.8833


[0.3933464288711548, 0.8833333253860474]